In [5]:
%load_ext autoreload
from quantifier import *
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%autoreload 2
l1df=readin()

In [9]:
l1df

,Lane,Band No.,Band Label,Mol. Wt. (KDa),Relative Front,Adj. Volume (Int),Volume (Int),Abs. Quant.,Rel. Quant.,Band %,Lane %
0,1,1,NaN,250,0.093333,36072,77688,NaN,NaN,1.725979,1.430039
1,1,2,NaN,150,0.181667,123984,205848,NaN,NaN,5.932408,4.915225
2,1,3,NaN,100,0.275000,42876,205416,NaN,NaN,2.051538,1.699777
3,1,4,NaN,75,0.350000,238140,350784,NaN,NaN,11.394564,9.440829
4,1,5,NaN,50,0.465000,381780,506736,NaN,NaN,18.267475,15.135297
5,1,6,NaN,37,0.553333,145008,235332,NaN,NaN,6.938368,5.748701
6,1,7,NaN,25,0.675000,736884,876060,NaN,NaN,35.258552,29.213050
7,1,8,NaN,20,0.720000,129636,215748,NaN,NaN,6.202846,5.139293
8,1,9,NaN,15,0.821667,125100,249084,NaN,NaN,5.985806,4.959468
9,1,10,NaN,10,0.895000,130464,272016,NaN,NaN,6.242464,5.172119
